In [1]:
#self add
from langchain.chat_models import init_chat_model
from langchain_ollama import OllamaEmbeddings
model = init_chat_model("gemma3:1b", model_provider="ollama")
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [2]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [4]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
with open('data/art15.json' , 'r') as f:
    data = json.dumps(f.read())
    docs = [Document(page_content=data, metadata={"source": "art15.json"})]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

Embed & Store Vectors

In [ ]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [ ]:
retriever = vector_store.as_retriever(
    search_type = "similarity",     # "mmr" or "similarity_score_threshold" also work
    search_kwargs = {"k": 4}
)


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

SYSTEM = """You are an expert legal assistant.
Answer *only* from the context between <context></context>;
if the answer isn’t there, say “I don't know.”"""
USER = """<context>\n{context}\n</context>\n\nQuestion: {input}"""

prompt = ChatPromptTemplate.from_messages([("system", SYSTEM), ("user", USER)])


combine_docs_chain = create_stuff_documents_chain(model, prompt)  


rag_chain = create_retrieval_chain(retriever, combine_docs_chain) #The Runnable return is a dictionary containing at the very least a context and answer key.

In [ ]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['InMemoryVectorStore', 'OllamaEmbeddings'], vectorstore=<langchain_core.vectorstores.in_memory.InMemoryVectorStore object at 0x7f17c7376e90>, search_kwargs={'k': 4}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an expert legal assistant.\nAnswer *only* from the context between <context></context>;\nif the answer isn’t th

In [ ]:
question = "Ajay does not allow a customer to enter his shop because he is a Muslim. Is this allowed under Article 15 of the Indian Constitution?" 
result   = rag_chain.invoke({"input": question})

print(result["answer"])      # grounded answer
print(result["context"])     # the stuffed context string (if you need sources, see below)


No, this is not allowed under Article 15 of the Indian Constitution. The text states: “No citizen shall, on grounds only of religion, race, caste, sex, or any of them, be subject to any disability, liability, restriction or condition with regard to —(a) Access to shops, public restaurants, hotels and places of public entertainment; or(b) The use of wells, tanks, bathing ghats, roads and places of public resort maintained wholly or partly out of State funds or dedicated to the use of.” The provision explicitly prohibits discrimination based on religion, race, caste, sex, or place of birth.
[Document(id='9dc11878-a2c6-42c2-9b20-ab2c77bcd609', metadata={'source': 'https://www.constitutionofindia.net/articles/article-15-prohibition-of-discrimination-on-grounds-of-religion-race-caste-sex-or-place-of-birth/'}, page_content='no citizen shall, on grounds only of religion, race, caste, sex or any of them, be subject to any disability, liability, restriction or condition with regard to-(a) Acces